## Realxed A-optimal experiment design

아래와 같이 생성된 데이터가 있습니다.

In [1]:
using Random
Random.seed!(2023)
n = 5; # 추정해야 하는 parameter의 개수
p = 20; # 가능한 실험의 가짓수
m = 30; # 실험 횟수
V = randn(n,p); #V의 각 열은 v1,...,vp 벡터를 나타냅니다.

In [2]:
V

5×20 Matrix{Float64}:
  0.100418  -0.205944    0.174579  …  -0.358438   -3.05665   -0.950952
  0.539314   0.910159    0.599171     -0.528173   -0.43402    0.214853
  0.032493  -0.7272     -0.219329     -0.0194823  -0.766573   1.9933
 -1.82519   -0.211406    0.892193      0.709027    0.085981   1.23662
 -1.1939    -0.0310205   1.07485      -0.21846    -0.726084   0.0255155

Q1. 다음과 같이 표현된 $\lambda \in \mathbb{R}^p$에 대한 optimization 문제를 해결하세요.

$$
\begin{array}{ll}
    \text{minimize}   & (1/m)\mathbf{tr}\left(\sum_{i=1}^p \lambda_i\nu_i\nu_i^T\right)^{-1} \\
    \text{subject to} & \mathbf{1}^T\lambda =1,\,\,\lambda\succeq 0
\end{array}
$$


Q2. 위 문제의 solution $\lambda^*$는 $m_1,\,m_2,\,...\,,m_p$에 대한 아래 A-optimal design problem의 solution의 lower bound가 됩니다. 그 이유는 무엇일까요? (세션의 relaxed experiment design 부분에서 나왔습니다!)

$$
\begin{array}{ll}
    \text{minimize}   & \mathbf{tr}\left(\sum_{i=1}^p m_i\nu_i\nu_i^T\right)^{-1} \\
    \text{subject to} & m_1 + m_2 +\,...\,+ m_p = m ,\,\, m \in \left\{0,1,\,...\,,m\right\}
\end{array}
$$

Q3. solution을 구한 후 $\hat{m}_i = \mathbf{round}(m\lambda_i^*)$를 구하세요. 이 정수 $\hat{m}_i$들에 대한 sub-optimal objective value를 계산하고, 이 upper bound와 lower bound의 차이를 구하세요. 

> 입력 변수로 주어진 행렬 Σ에 대한 A-최적설계(A-optimal design)를 구하는 문제에서 Σ의 역행렬이 최대가 되어야 한다.A-최적설계는 주로 최소분산추정량(Minimum variance estimation)의 효율성을 최대화하기 위해 사용된다. 즉, A-최적설계에서는 특정한 실험을 최소한으로 수행하여 실험 결과에 대한 정보를 최대한 많이 얻을 수 있도록 설계하는 것이다.

> 주어진 실험 계획에 대한 정보가 행렬 Σ에 포함되어 있으며, 최적화 문제에서 구한 λ* 는 행렬 Σ의 조합을 결정하는 값이다.

> 따라서, 최적화 문제에서 구한 λ는 실험계획의 A-최적설계를 나타내며, λ의 역수인 (∑𝑝𝑖=1𝜆𝑖𝜈𝑖𝜈𝑇𝑖)^-1은 A-최적설계의 행렬 Σ의 역행렬에 해당한다. 그래서 주어진 최적화 문제에서 구한 λ* 는 A-최적설계의 하한선(lower bound)이 된다.

> 주어진 실험 계획에 대한 정보가 행렬 Σ에 포함되어 있으며, 최적화 문제에서 구한 λ*는 행렬 Σ의 조합을 결정하는 값이다.

#유의사항: Convex 패키지는 trace로 구성된 목적함수를 인식하지 못하므로, matrix fractional function (교재 76p example 3.4 참조)을 이용하여 formulation해줘야 합니다. kth unit vector $e_k$에 대해 reformulation하면 위의 목적함수는 다음과 같이 바뀝니다. 사실 trace를 다른 형태로 표현한 것에 불과하다는 점이 보입니다.

$$ (1/m)\sum_{k=1}^n e_k^T \left(\sum_{i=1}^p \lambda_i\nu_i\nu_i^T\right)^{-1}e_k $$

#hint: 벡터 x, 행렬 P에 대해 함수 matrixfrac(x, P)는 $x^TP^{-1}x$를 리턴합니다.

In [3]:
using Convex, SCS
using LinearAlgebra

In [4]:
#obj 함수 정의와 constraints를 채우세요.
λ = Variable(p)  
obj = 0
mat1 = zeros(5,5)
for k in 1:n #바깥쪽 k에 대한 시그마의 표현
    ek = zeros(n,1) #kth unit vector를 표현하는 것입니다.
    ek[k] = 1
    for i in 1:20
        mat1 += λ[i]*V[:,i]*transpose(V[:,i])
    end
    obj += 1/m * matrixfrac(ek,mat1)
end 

problem = minimize(obj)

problem.constraints += sum(λ)==1
problem.constraints += λ >= 0

solve!(problem, SCS.Optimizer)

sol_lambda = evaluate(λ);

------------------------------------------------------------------
	       SCS v3.2.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 26, constraints m: 202
cones: 	  z: primal zero / dual free vars: 77
	  l: linear vars: 20
	  s: psd vars: 105, ssize: 5
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 1551, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|3.12e+001 1.00e+000 2.63e+000 -1.02e+000 1.00e-001 1.17e-003 
   150|7.80e-005 1.90e-004 3.89e-005 6.84e-002 1.00e-001 2.96e-002

In [5]:
evaluate(λ)

20-element Vector{Float64}:
  0.10222400214619148
 -1.700604255726217e-6
 -6.057339206654259e-6
  0.14205185623223834
 -2.630805203447584e-6
 -3.8365792859731184e-7
 -8.922599895275597e-6
  1.624325538214939e-6
  0.0811229004734365
 -2.185353287862209e-6
 -6.306031921437921e-6
 -3.2527405429452482e-6
  0.23077235724503706
  9.311625982721185e-7
 -2.5804391835543675e-7
  0.1387184969624399
  0.023833210974340758
 -2.0087382623913585e-6
  0.2001894832348481
  0.0811188660274495

In [6]:
m_hat = round.(m*sol_lambda)

20-element Vector{Float64}:
  3.0
 -0.0
 -0.0
  4.0
 -0.0
 -0.0
 -0.0
  0.0
  2.0
 -0.0
 -0.0
 -0.0
  7.0
  0.0
 -0.0
  4.0
  1.0
 -0.0
  6.0
  2.0

In [7]:
k = zeros(5,5)

5×5 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [8]:
for i in 1:20
    k += sol_lambda[i] * V[:,i] * transpose(V[:,i])
end

In [9]:
tr(k)

6.57391184426424

In [10]:
subopt = tr(k)

6.57391184426424

In [11]:
subopt - evaluate(obj)

6.505522340283494